## **Mount Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from tensorflow.keras.models import load_model
model = load_model("/content/drive/MyDrive/QR_Intern/cnn_price_trend_model.h5")

In [3]:
import os

BASE_DIR = "/content/drive/MyDrive/QR_Intern"
os.makedirs(BASE_DIR, exist_ok=True)

# **Load Data**

In [4]:
import pandas as pd

df = pd.read_csv(f"{BASE_DIR}/data.csv", parse_dates=["Date"])

# **Load the Trained Model**

In [5]:
from tensorflow.keras.models import load_model

model_path = f"{BASE_DIR}/cnn_price_trend_model.h5"
model = load_model(model_path)

# **Prepare Backtesting Data Format**

In [6]:
bt_df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()
bt_df['EMA20'] = df['EMA20'].values
bt_df['MACD'] = df['MACD'].values
bt_df['MOM_6'] = df['MOM_6'].values
bt_df['RSI'] = df['RSI'].values
bt_df['ATR'] = df['ATR'].values

bt_df.set_index('Date', inplace=True)

# **Define Strategy**

In [12]:
# -*- coding: utf-8 -*-
"""Backtesting Script for Stock Trend Prediction"""

# 1. Install required packages
import subprocess
import sys
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    from backtesting import Strategy, Backtest
except ImportError:
    install('backtesting')
    from backtesting import Strategy, Backtest

import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

# 2. Suppress unnecessary warnings
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='backtesting._plotting')
warnings.filterwarnings('ignore', category=UserWarning, module='absl')

# 3. Load data and model
BASE_DIR = "/content/drive/MyDrive/QR_Intern"
try:
    model = load_model(f"{BASE_DIR}/cnn_price_trend_model.h5")
    df = pd.read_csv(f"{BASE_DIR}/data.csv", parse_dates=["Date"], index_col="Date")
except Exception as e:
    print(f"Error loading files: {e}")
    raise

# 4. Define Strategy
class SlopeDetectionStrategy(Strategy):
    window = 22  # As per research paper
    confidence_thresh = 0.65  # Min probability to trade

    def init(self):
        self.model = model
        # Commission is now set in Backtest() initialization

    def next(self):
        if len(self.data) < self.window:
            return

        # Prepare input tensor
        X = np.column_stack([
            self.data.EMA20[-self.window:],
            self.data.MACD[-self.window:],
            self.data.MOM_6[-self.window:],
            self.data.RSI[-self.window:],
            self.data.ATR[-self.window:]
        ]).reshape(1, self.window, 5)

        # Get prediction
        pred = self.model.predict(X, verbose=0)[0]
        label = np.argmax(pred)
        confidence = np.max(pred)

        # Trading rules (4-class)
        if confidence > self.confidence_thresh:
            if label in [2, 3]:  # rise or rise-plus
                if not self.position:
                    self.buy(size=0.5)  # Risk management
            elif label in [0, 1]:  # fall or fall-plus
                if self.position:
                    self.position.close()

# 5. Run backtest
bt = Backtest(
    df,
    SlopeDetectionStrategy,
    cash=10000,
    commission=0.002,
    exclusive_orders=True,
    trade_on_close=True
)

stats = bt.run()
print(stats)
bt.plot()

Backtest.run:   0%|          | 0/216 [00:00<?, ?bar/s]

Start                     2024-08-20 00:00:00
End                       2025-07-02 00:00:00
Duration                    316 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                      10000.0
Equity Peak [$]                       10000.0
Return [%]                                0.0
Buy & Hold Return [%]                12.43562
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
CAGR [%]                                  0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Alpha [%]                                 0.0
Beta                                      0.0
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                      

GridPlot(id='p1509', ...)

# **Save Results to CSV**

In [14]:
# 6. Save results
stats_df = pd.DataFrame([stats])
try:
    stats_df.to_csv(f"{BASE_DIR}/backtest_results.csv", index=False)
    print(f"Results saved to {BASE_DIR}/backtest_results.csv")
except Exception as e:
    print(f"Error saving results: {e}")

Results saved to /content/drive/MyDrive/QR_Intern/backtest_results.csv
